<h1>Using a Pre-Trained VGG Model for Image Classification with CIFAR-10</h1>

<h5>Import libraries</h5>

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader
from torch.utils.data import Subset

<h5>Data Loading and Transformation:</h5>
<ul>
<li><b>Training Set:</b> Images are resized to 224x224 (to match VGG input size), randomly cropped with padding, and horizontally flipped for augmentation. Normalization is done using CIFAR-10 statistics.</li>
<li><b>Testing Set:</b> Images are resized to 224x224 and normalized similarly to the training set.</li>
</ul>

In [12]:
# Define transformations for training and testing
transform_train = transforms.Compose([
    transforms.Resize(224),  # Resize images to 224x224
    transforms.RandomCrop(224, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.Resize(224),  # Resize images to 224x224
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

<h5>Data Downloading and Loading:</h5>

CIFAR-10 dataset is downloaded and loaded into training and testing sets using DataLoader.

In [14]:
# Download and prepare CIFAR-10 dataset
full_trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)

indices = list(range(100)) # making subset of 100 images for train-data

trainset = Subset(full_trainset,indices)
trainloader = DataLoader(trainset, batch_size=10, shuffle=True, num_workers=2)

# test dataset
full_testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

indices = list(range(10))  # making subset of 10 images for test-data
testset = Subset(full_testset,indices)
testloader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


<h5>Model Loading and Modification:</h5>

<b>VGG-16 model pre-trained</b> on ImageNet is loaded.
The final layer of the classifier is replaced to match the number of classes in CIFAR-10 (10 classes).

In [16]:
# Load the pre-trained VGG model
model = models.vgg16(pretrained=True)

# Modify the model for CIFAR-10 (10 classes)
num_classes = 10
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

<h5>Model Training:</h5>

<b>Loss function (CrossEntropyLoss)</b> and <b>optimizer (SGD with momentum and weight decay)</b> are defined.<br>
The model is trained for a specified number of epochs. For each batch, the gradients are zeroed, the forward pass is computed, loss is calculated, backpropagation is performed, and weights are updated.

In [18]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)

In [20]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        running_loss += loss.item()
    
    # Print average loss for the epoch
    avg_loss = running_loss / len(trainloader)
    print(f'End of Epoch {epoch + 1}, Average Loss: {avg_loss:.4f}')

print('Finished Training')

End of Epoch 1, Average Loss: 2.2486
End of Epoch 2, Average Loss: 1.5038
End of Epoch 3, Average Loss: 1.0453
End of Epoch 4, Average Loss: 0.6159
End of Epoch 5, Average Loss: 0.5870
End of Epoch 6, Average Loss: 0.3064
End of Epoch 7, Average Loss: 0.1548
End of Epoch 8, Average Loss: 0.1708
End of Epoch 9, Average Loss: 0.2210
End of Epoch 10, Average Loss: 0.2264
Finished Training


<h5>Model Evaluation:</h5>

In [22]:
# Evaluate the model
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10 test images: {100 * correct / total:.2f}%')


Accuracy of the network on the 10 test images: 50.00%
